A CNN created with Flux.jl

In [6]:
using Pkg
Pkg.activate(".")

  Activating project at `~/LocalDocuments/Studia/AlgorytmyWInzynieriiDanych/KM3_Repo/examples/ReferenceSolution_Julia`


In [7]:
using JLD2
X_train = load("../../data/imdb_dataset_prepared.jld2", "X_train")
y_train = load("../../data/imdb_dataset_prepared.jld2", "y_train")
X_test = load("../../data/imdb_dataset_prepared.jld2", "X_test")
y_test = load("../../data/imdb_dataset_prepared.jld2", "y_test")
embeddings = load("../../data/imdb_dataset_prepared.jld2", "embeddings")
vocab = load("../../data/imdb_dataset_prepared.jld2", "vocab")
nothing

embedding_dim = size(embeddings,1)

50

In [18]:
display(vocab)

12849-element Vector{String}:
 "confined"
 "dumber"
 "henry"
 "abducted"
 "rises"
 "progression"
 "il"
 "gathered"
 "lovers"
 "cannibalistic"
 ⋮
 "poetic"
 "ponderous"
 "maybe"
 "towel"
 "uncut"
 "joint"
 "treacherous"
 "dev"
 "<pad>"

In [15]:
embeddings

50×12849 Matrix{Float32}:
  0.90951   -0.58014    0.27137   0.68397   …   0.45505     0.014323  0.0
 -0.20702   -1.1316     0.61347  -0.68729      -0.0014904  -0.74624   0.0
 -0.090611   0.44189   -0.52498   0.8797       -0.45487     0.35701   0.0
 -0.63721   -0.048199  -0.7617   -0.35249      -0.15543     0.75488   0.0
  0.051387  -0.11754    0.37252   0.82288      -1.2866      0.11551   0.0
 -0.26292    0.97308    0.21401  -0.17179   …  -0.10727    -0.37074   0.0
  0.14454    1.0075    -1.0817   -1.4887        0.37509     0.80859   0.0
  0.40134   -1.2014     0.16501   0.98021       0.85616    -0.64355   0.0
  0.17305   -1.2752    -0.45105   0.031865     -0.045315   -0.63822   0.0
 -0.23503    0.66842   -0.77013  -0.7007       -0.3152      0.74175   0.0
  ⋮                                         ⋱                         
 -0.2502     0.54529    0.8323   -0.28752      -0.057228    0.36188   0.0
  0.37538    0.68665   -0.63336   1.0756        1.1033     -0.79495   0.0
  1.1403     0.

In [8]:
using Flux

model = Chain(
    Flux.Embedding(length(vocab), embedding_dim),
    x->permutedims(x, (2,1,3)),
    Conv((3,), embedding_dim => 8, relu),
    MaxPool((8,)),
    Flux.flatten,
    Dense(128, 1, σ)
)

Chain(
  Embedding(12849 => 50),               # 642_450 parameters
  var"#13#14"(),
  Conv((3,), 50 => 8, relu),            # 1_208 parameters
  MaxPool((8,)),
  Flux.flatten,
  Dense(128 => 1, σ),                   # 129 parameters
)                   # Total: 5 arrays, 643_787 parameters, 2.456 MiB.

In [9]:
# add Glove embeddings to Embedding layer
model.layers[1].weight .= embeddings;

In [37]:
# Create a simple model with just the embedding layer
simple_model = Chain(Flux.Embedding(length(vocab), embedding_dim))
simple_model.layers[1].weight .= embeddings

# Take a sample input - first sequence from training data
sample_input = X_train[:,1]
println("Sample input (token indices): ", sample_input[1:10])  # Show first 10 tokens
println(summary(sample_input))

# Pass through embedding layer
embedded_output = simple_model(sample_input)
embedded_output

Sample input (token indices): [6391, 143, 286, 3838, 1529, 2329, 10682, 8101, 7377, 1934]
130-element Vector{Int64}


50×130 Matrix{Float32}:
  0.16265    1.1607   0.0   0.44285   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.37274    0.47395  0.0   0.31399      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.022479   1.3185   0.0   0.20969      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.50465    0.39748  0.0  -0.39017      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.27456   -0.45394  0.0   0.37531      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.093199   0.12834  0.0  -0.53613   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.73615   -0.4484   0.0  -0.9145       0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.95033   -0.13     0.0   0.88797      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.54603    0.21566  0.0  -0.054453     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.15323    0.24585  0.0   0.14011      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  ⋮                                   ⋱            ⋮                   
 -0.52859   -0.12209  0.0   0.046413     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.40937    0.27572  0.0  -0.2176       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.

In [10]:
using Printf, Statistics

dataset = Flux.DataLoader((X_train, y_train), batchsize=64, shuffle=true)

loss(m, x, y) = Flux.Losses.binarycrossentropy(m(x), y)
accuracy(m, x, y) = mean((m(x) .> 0.5) .== (y .> 0.5))

opt = Optimisers.setup(Adam(), model)

epochs = 5
for epoch in 1:epochs
    total_loss = 0.0
    total_acc = 0.0
    num_samples = 0

    t = @elapsed begin
        for (x, y) in dataset
            grads = Flux.gradient(model) do m
                loss(m, x, y)
            end
            Optimisers.update!(opt, model, grads[1])
            total_loss += loss(model, x, y)
            total_acc += accuracy(model, x, y)
            num_samples += 1
        end

        train_loss = total_loss / num_samples
        train_acc = total_acc / num_samples

        test_acc = accuracy(model, X_test, y_test)
        test_loss = loss(model, X_test, y_test)
    end

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)", 
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

Epoch: 1 (35.34s) 	Train: (l: 0.54, a: 0.73) 	Test: (l: 0.40, a: 0.82)
Epoch: 2 (13.29s) 	Train: (l: 0.33, a: 0.86) 	Test: (l: 0.34, a: 0.86)
Epoch: 3 (14.54s) 	Train: (l: 0.25, a: 0.90) 	Test: (l: 0.32, a: 0.87)
Epoch: 4 (13.25s) 	Train: (l: 0.19, a: 0.93) 	Test: (l: 0.33, a: 0.86)
Epoch: 5 (13.39s) 	Train: (l: 0.14, a: 0.95) 	Test: (l: 0.35, a: 0.86)
